In [117]:
import pandas as pd
import json
import networkx as nx
from networkx_query import search_nodes, search_edges
import numpy as np
import math

In [118]:
with open('/Users/theojolliffe/Documents/Wayback BBC/topicScoresOrder.json', 'r') as fp:
    topicScoresOrder = json.load(fp)

topicScoresOrder = {i: {i[0]: 1/(1+i[1]/10) for i in topicScoresOrder[i]} for i in topicScoresOrder}

for i in topicScoresOrder.keys():
    topicScoresOrder[i]['welsh'] = 0.1
    topicScoresOrder[i]['disability'] = 0.1

In [119]:
topicScoresOrder

{'cumbria': {'travel': 0.7278796788254589,
  'health': 0.7373155446935475,
  'children': 0.7476531793275849,
  'care': 0.7585366002252635,
  'hoursworked': 0.7611622171779674,
  'economic': 0.7636918415774013,
  'marital': 0.7685554019510524,
  'tenure': 0.779938758790283,
  'household': 0.7882348325426418,
  'population': 0.7963740295092272,
  'ethnicity': 0.7982391433758176,
  'agemed': 0.7992360398420778,
  'religion': 0.7995554267899909,
  'welsh': 0.1,
  'disability': 0.1},
 'lancashire': {'travel': 0.7171315720239612,
  'children': 0.748713647791873,
  'care': 0.7513147822620997,
  'health': 0.7586229979282538,
  'household': 0.7757078208924982,
  'tenure': 0.7759788872262368,
  'agemed': 0.7806512760808537,
  'religion': 0.7820732983808982,
  'marital': 0.7852504244841253,
  'economic': 0.7873235362629281,
  'population': 0.787423350003117,
  'hoursworked': 0.7972097949882635,
  'ethnicity': 0.8084732213155831,
  'welsh': 0.1,
  'disability': 0.1},
 'merseyside': {'travel': 0.71

In [120]:
# COLLECT 2021 LAs

las2021 = pd.read_csv('./pipeline_active/lookup/lad2015_lad2021.csv')
las2021 = [x for x in las2021['parent'] if str(x) != 'nan']
las2021 = list(set(las2021))

lads = []
for i in las2021:
    try:
        lads.append(json.load(open(f'/Users/theojolliffe/Documents/pipeline_active/final/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
        print(i)
        # pass

# Remove tiny areas
# lads = [lad for lad in lads if (lad['name'] != 'City of London')&(lad['name'] != 'Isle of Scilly')]

for lad in lads:
    lad['data']['agemed']['value']['change']['all'] = lad['data']['agemed']['value']['2011']['all']-lad['data']['agemed']['value']['2001']['all']
    lad['data']['density']['value']['change']['all'] = lad['data']['density']['value']['2011']['all']-lad['data']['density']['value']['2001']['all']

In [121]:
options = pd.read_csv('./csv/lists/places_2020.csv')
areaType = pd.read_csv('../census-data-transformed/csv/lists/censusAreaLookup.csv')
typeLookup = {}
for i in areaType.index:
    typeLookup[areaType.iloc[i]['Name']]=areaType.iloc[i]['Group name']

areas = []
for i in options["code"]:
    try:
        areas.append(json.load(open(f'/Users/theojolliffe/Documents/pipeline_active/final/json/place/{i}.json', 'rb')))
    except FileNotFoundError:
#         print(i)
        pass

for area in areas:
    area['data']['agemed']['value']['change']['all'] = area['data']['agemed']['value']['2011']['all']-area['data']['agemed']['value']['2001']['all']
    area['data']['density']['value']['change']['all'] = area['data']['density']['value']['2011']['all']-area['data']['density']['value']['2001']['all']

# Seperate the areas by area type
regions = []
for i in areas:
    if i['type']=='rgn':
        regions.append(i)

countries = []
for i in areas:
    if (i['type']=='ew')|(i['type']=='ctry'):
        countries.append(i)

In [122]:
for lad in lads:
    # lad['data']['disability'] = lad['data']['care'].copy()
    lad['data']['disability'] = {}

    for k in lad['data']['care'].keys():
        lad['data']['disability'][k] = {}
        for j in lad['data']['care'][k].keys():
            lad['data']['disability'][k][j] = {}
            lad['data']['disability'][k][j]['notDisabled'] = lad['data']['care'][k][j]['noCare']
            lad['data']['disability'][k][j]['little'] = lad['data']['care'][k][j]['1to19hoursWeek']
            if 'rank' in k:
                lad['data']['disability'][k][j]['lot'] = lad['data']['care'][k][j]['20to49hoursWeek']
            else:
                lad['data']['disability'][k][j]['lot'] = lad['data']['care'][k][j]['20to49hoursWeek'] + lad['data']['care'][k][j]['40PlushoursWeek']

            # del lad['data']['disability'][k][j]['noCare']
            # del lad['data']['disability'][k][j]['1to19hoursWeek']
            # del lad['data']['disability'][k][j]['20to49hoursWeek']
            # del lad['data']['disability'][k][j]['40PlushoursWeek']

for lad in countries:
    # lad['data']['disability'] = lad['data']['care'].copy()
    lad['data']['disability'] = {}

    for k in lad['data']['care'].keys():
        lad['data']['disability'][k] = {}
        for j in lad['data']['care'][k].keys():
            lad['data']['disability'][k][j] = {}
            lad['data']['disability'][k][j]['notDisabled'] = lad['data']['care'][k][j]['noCare']
            lad['data']['disability'][k][j]['little'] = lad['data']['care'][k][j]['1to19hoursWeek']
            if 'rank' in k:
                lad['data']['disability'][k][j]['lot'] = lad['data']['care'][k][j]['20to49hoursWeek']
            else:
                lad['data']['disability'][k][j]['lot'] = lad['data']['care'][k][j]['20to49hoursWeek'] + lad['data']['care'][k][j]['40PlushoursWeek']

            # del lad['data']['disability'][k][j]['noCare']
            # del lad['data']['disability'][k][j]['1to19hoursWeek']
            # del lad['data']['disability'][k][j]['20to49hoursWeek']
            # del lad['data']['disability'][k][j]['40PlushoursWeek']

for lad in regions:
    # lad['data']['disability'] = lad['data']['care'].copy()
    lad['data']['disability'] = {}

    for k in lad['data']['care'].keys():
        lad['data']['disability'][k] = {}
        for j in lad['data']['care'][k].keys():
            lad['data']['disability'][k][j] = {}
            lad['data']['disability'][k][j]['notDisabled'] = lad['data']['care'][k][j]['noCare']
            lad['data']['disability'][k][j]['little'] = lad['data']['care'][k][j]['1to19hoursWeek']
            if 'rank' in k:
                lad['data']['disability'][k][j]['lot'] = lad['data']['care'][k][j]['20to49hoursWeek']
            else:
                lad['data']['disability'][k][j]['lot'] = lad['data']['care'][k][j]['20to49hoursWeek'] + lad['data']['care'][k][j]['40PlushoursWeek']

            # del lad['data']['disability'][k][j]['noCare']
            # del lad['data']['disability'][k][j]['1to19hoursWeek']
            # del lad['data']['disability'][k][j]['20to49hoursWeek']
            # del lad['data']['disability'][k][j]['40PlushoursWeek']


In [123]:
# Transform the welsh language data

for area in lads:
    if area['parents'][0]['name']=='Wales':
        x = area['data']['welsh']
        for y in x.keys():
            if y=='value':
                for z in x[y].keys():
                    if z=='change':
                        x[y][z]['NoWelsh'] = round(((x[y]['2011']['NoWelsh'] - x[y]['2001']['NoWelsh'])/x[y]['2001']['NoWelsh'])*100, 2)
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']
                    else:
                        x[y][z]['NoWelsh'] = x[y][z]['AllOver3'] - x[y][z]['SpeaksWelsh']
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']

for area in lads:
    if area['parents'][0]['name']=='Wales':
        x = area['data']['welsh']
        for y in x.keys():
            if y=='perc':
                for z in x[y].keys():
                    if z=='change':
                        x[y][z]['NoWelsh'] = x[y]['2011']['NoWelsh'] - x[y]['2001']['NoWelsh']
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']
                        for w in x[y][z].keys():
                            x[y][z][w] = round(x[y]['2011'][w]-x[y]['2001'][w], 2)
                    else:
                        x[y][z]['NoWelsh'] = x[y][z]['AllOver3'] - x[y][z]['SpeaksWelsh']
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']
                        for w in x[y][z].keys():
                            x[y][z][w] = round((x[y][z][w]/x[y][z]['all'])*100, 2)

for area in lads:
    if area['parents'][0]['name']!='Wales':
        x = area['data']['welsh']
        for y in x.keys():
            for z in x[y].keys():
                x[y][z]['all'] = 1
                del x[y][z]['AllOver3']
                for w in x[y][z].keys():
                    x[y][z][w] = x[y][z][w] = 1

In [124]:
for area in lads:
    x = area['data']
    for a in x.keys():
        for y in x[a].keys():
            for z in x[a][y].keys():
                for b in x[a][y][z].keys():
                    if math.isinf(x[a][y][z][b]):
                        x[a][y][z][b] = 'Inf'

In [125]:
# Transform the welsh language data

for area in countries:
    if area['name']=='Wales':
        x = area['data']['welsh']
        for y in x.keys():
            if y=='value':
                for z in x[y].keys():
                    if z=='change':
                        x[y][z]['NoWelsh'] = round(((x[y]['2011']['NoWelsh'] - x[y]['2001']['NoWelsh'])/x[y]['2001']['NoWelsh'])*100, 2)
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']
                    else:
                        x[y][z]['NoWelsh'] = x[y][z]['AllOver3'] - x[y][z]['SpeaksWelsh']
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']

for area in countries:
    if area['name']=='Wales':
        x = area['data']['welsh']
        for y in x.keys():
            if y=='perc':
                for z in x[y].keys():
                    if z=='change':
                        x[y][z]['NoWelsh'] = x[y]['2011']['NoWelsh'] - x[y]['2001']['NoWelsh']
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']
                        for w in x[y][z].keys():
                            x[y][z][w] = round(x[y]['2011'][w]-x[y]['2001'][w], 2)
                    else:
                        x[y][z]['NoWelsh'] = x[y][z]['AllOver3'] - x[y][z]['SpeaksWelsh']
                        x[y][z]['all'] = x[y][z]['AllOver3']
                        del x[y][z]['AllOver3']
                        for w in x[y][z].keys():
                            x[y][z][w] = round((x[y][z][w]/x[y][z]['all'])*100, 2)
        
for area in countries:
    if area['name']!='Wales':
        x = area['data']['welsh']
        for y in x.keys():
            for z in x[y].keys():
                x[y][z]['all'] = 1
                del x[y][z]['AllOver3']
                for w in x[y][z].keys():
                    x[y][z][w] = x[y][z][w] = 1


In [126]:
nameChanges = {
    "Herefordshire, County of": "Herefordshire",
    "Bristol, City of": "Bristol",
    # "City of London": "the City of London",
    # "Isles of Scilly": "the Isles of Scilly",
    # "Isle of Anglesey": "the Isle of Anglesey",
    "Derbyshire Dales": "the Derbyshire Dales",
    "Kingston upon Hull, City of": "Kingston upon Hull",
    # "New Forest": "the New Forest"
}
for lad in lads:
    if lad['name'] in nameChanges.keys():
        lad['name'] = nameChanges[lad['name']]

In [127]:
simiDF = pd.read_csv('/Users/theojolliffe/Documents/correspondinglocalauthoritiesv3r.csv')

In [128]:
noOfLads = {"England": 309, "Wales": 22}
areaClassDF = pd.read_csv('../census-data-transformed/csv/lists/censusAreaLookup.csv')
areaClassDF.at[75, 'Code'] = "E06000058"
areaClassLU = {
    "1a1r": "affluent suburban area", "1b1r": "affluent rural area", 
    "1b2r": "growing rural area", "2a1r": "city", 
    "2b1r": "area", "3a2r": "remote area", 
    "3a1r": "agricultural area", "3b1r": "coastal area",
    "3b2r": "seaside  area",
    "4a1r": "urban area", "5a1r": "cosmopolitan area",
    "6a2r": "industrial area", "6a3r": "service economy area",
    "6a1r": "industrial area", "7a1r": "rural area",
    "7c2r": "affluent area", "7c1r": "affluent area",
    "8a1r": "multi-cultural area", "8a2r": "urban area",
    "8b1r": "suburban area", "8b2r": "suburban area",
}
areaClassSuperLU = {
    "1r": "affluent area",
    "2r": "urban area",
    "3r": "rural area",
    "4r": "multi-cultural area",
    "5r": "cosmopolitan area",
    "6r": "industrial area",
    "7r": "suburban area",
    "8r": "built up area",
}
gssLookup = {"E09": "London borough", "E08": "metropolitan district", "E07": "district", "E06": "unitary authority", "W06": "Welsh district"}
gssLookupShort = {"E09": "borough", "E08": "district", "E07": "district", "E06": "unitary authority", "W06": "district"}

In [129]:
bbcReg = pd.read_csv('/Users/theojolliffe/Downloads/bbc_regions.csv')
bbcRegLU = {}
for i in bbcReg.index:
    if bbcReg['areaCode'].iloc[i] in [lad['code'] for lad in lads]:
        bbcRegLU[bbcReg['areaCode'].iloc[i]] = bbcReg['bbc_online'].iloc[i]
bbcRegLU['E06000053'] = 'cornwall'
bbcRegLU['E06000060'] = 'beds_bucks_and_herts'
bbcRegLU['E09000001'] = 'london'
bbcRegLU['E07000150'] = 'northamptonshire'
bbcRegLU['E06000061'] = 'northamptonshire'
bbcRegLU['E06000062'] = 'northamptonshire'

regionsBBC = ['cumbria', 'lancashire', 'merseyside', 'manchester', 'tees', 'tyne_and_wear', 'Humberside', 'york_and_north_yorkshire', 'leeds_and_west_yorkshire', 'lincolnshire', 'south_yorkshire', 'birmingham_and_black_country', 'coventry_and_warwickshire', 'hereford_and_worcester', 'shropshire', 'stoke_and_staffordshire', 'derbyshire', 'leicester', 'northamptonshire', 'nottingham', 'bristol', 'cornwall', 'devon', 'gloucestershire', 'somerset', 'wiltshire', 'beds_bucks_and_herts', 'cambridgeshire', 'essex', 'norfolk', 'suffolk', 'berkshire', 'dorset', 'hampshire', 'oxford', 'kent', 'london', 'surrey', 'sussex']
bbcNames = {
     'derby': 'derbyshire', 
     'hereford and worcester': 'hereford_and_worcester',
     'tyne and wear': 'tyne_and_wear',
     'northampton': 'northamptonshire',
     'liverpool': 'merseyside',
     'coventry and warwickshire': 'coventry_and_warwickshire',
     'humberside': 'Humberside',
     'sheffield and south yorkshire': 'south_yorkshire',
     'hampshire & isle of wight': 'hampshire',
     'stoke and staffordshire': 'stoke_and_staffordshire',
     'york & north yorkshire': 'york_and_north_yorkshire',
     'birmingham and black country': 'birmingham_and_black_country',
     'beds, bucks and herts': 'beds_bucks_and_herts',
     'leeds and west yorkshire': 'leeds_and_west_yorkshire'
}
for i in bbcRegLU.keys():
    bbcRegLU[i] = bbcRegLU[i].lower()
    if bbcRegLU[i] not in regionsBBC:
        try:
            bbcRegLU[i] = bbcNames[bbcRegLU[i]]
        except:
            pass

In [130]:
tfidf = pd.read_csv('/Users/theojolliffe/Documents/Wayback BBC/BBCRegionalTFIDF.csv')
tfidf = tfidf.set_index('Unnamed: 0')
for i in tfidf.columns:
    for j in tfidf[i].index:
        tfidf[i].loc[j] = 1+100*tfidf[i].loc[j]
tfidfLU = {}
for i in regionsBBC:
    tfidfLU[i]=tfidf.loc[i].to_dict()

In [131]:
TFtopicLU = {
    'health': 'health',
    'racial': 'ethnicity',
    'population': 'population',
    'age': 'agemed',
    'work': 'economic',
    'commuter': 'travel',
    'housing': 'tenure'
}
TFtopicRev = dict((TFtopicLU[x],x) for x in TFtopicLU)

In [132]:
def this(lad):
    return [l for l in lads if l['name']==lad][0]

In [133]:
topics = [{1: 'population', 2: 'value', 3: 'all'}, 
          {1: 'density', 2: 'value', 3: 'all'},
          {1: 'agemed', 2: 'value', 3: 'all'},
          {1: 'welsh', 2: 'perc', 3: 'SpeaksWelsh'},
          {1: 'welsh', 2: 'perc', 3: 'all'},
          {1: 'care', 2: 'perc', 3: '20to49hoursWeek'},
          {1: 'care', 2: 'perc', 3: '40PlushoursWeek'},
          {1: 'disability', 2: 'perc', 3: 'notDisabled'},
          {1: 'disability', 2: 'perc', 3: 'little'},
          {1: 'disability', 2: 'perc', 3: 'lot'},
          {1: 'children', 2: 'perc', 3: 'NoKids'},
          {1: 'children', 2: 'perc', 3: 'Kids'},
          {1: 'children', 2: 'perc', 3: 'NonDepKids'},
          {1: 'tenure', 2: 'perc', 3: 'rented_social'},
          {1: 'tenure', 2: 'perc', 3: 'rented_private'},
          {1: 'tenure', 2: 'perc', 3: 'owned'},
          {1: 'religion', 2: 'perc', 3: 'Hindu'},
          {1: 'religion', 2: 'perc', 3: 'Buddhist'},
          {1: 'religion', 2: 'perc', 3: 'Sikh'},
          {1: 'religion', 2: 'perc', 3: 'Jewish'},
          {1: 'religion', 2: 'perc', 3: 'Christian'},
          {1: 'religion', 2: 'perc', 3: 'Muslim'},
          {1: 'religion', 2: 'perc', 3: 'Noreligion'},
          {1: 'religion', 2: 'perc', 3: 'Otherreligion'},
          {1: 'household', 2: 'perc', 3: 'OnePerson'},
          {1: 'household', 2: 'perc', 3: 'Married'},
          {1: 'household', 2: 'perc', 3: 'LoneParent'},
          {1: 'household', 2: 'perc', 3: 'Cohabiting'},
          {1: 'health', 2: 'perc', 3: 'bad'},
          {1: 'health', 2: 'perc', 3: 'good'},
          {1: 'ethnicity', 2: 'perc', 3: 'black'},
          {1: 'ethnicity', 2: 'perc', 3: 'white'},
          {1: 'ethnicity', 2: 'perc', 3: 'asian'},
          {1: 'ethnicity', 2: 'perc', 3: 'other'},
          {1: 'ethnicity', 2: 'perc', 3: 'mixed'},
          {1: 'marital', 2: 'perc', 3: 'Married'},
          {1: 'marital', 2: 'perc', 3: 'Seperated'},
          {1: 'marital', 2: 'perc', 3: 'Single'},
          {1: 'hoursworked', 2: 'perc', 3: 'Female1-15'},
          {1: 'hoursworked', 2: 'perc', 3: 'Male1-15'},
          {1: 'hoursworked', 2: 'perc', 3: 'Female49plus'},
          {1: 'hoursworked', 2: 'perc', 3: 'Male49plus'},
          {1: 'economic', 2: 'perc', 3: 'employee'},
          {1: 'economic', 2: 'perc', 3: 'student'},
          {1: 'economic', 2: 'perc', 3: 'unemployed'},
          {1: 'economic', 2: 'perc', 3: 'self-employed'}]

ladCodes = []
for lad in lads:
    ladCodes.append(lad['code'])
    
    
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'topic', 'parent'])
    for i in range(len(lads)):
        print(topics[j][1])
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        df1['parent'].iloc[i] = lads[i]['parents'][0]['name']
    df = pd.concat([df,df1])


population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population
population

In [134]:
df[df['topic'].map(lambda x: x.split("_")[1] == 'Sikh')].sort_values(['change'])

,lad,2001,2011,change,topic,parent
E09000011,Greenwich,2.19,1.37,-0.82,religion_Sikh,London
E09000025,Newham,2.83,2.08,-0.75,religion_Sikh,London
E09000009,Ealing,8.51,7.91,-0.6,religion_Sikh,London
E09000005,Brent,0.66,0.55,-0.11,religion_Sikh,London
E09000031,Waltham Forest,0.57,0.47,-0.1,religion_Sikh,London
...,...,...,...,...,...,...
E08000031,Wolverhampton,7.58,9.09,1.51,religion_Sikh,West Midlands
E06000039,Slough,9.09,10.62,1.53,religion_Sikh,South East
E08000028,Sandwell,6.87,8.74,1.87,religion_Sikh,West Midlands
E09000017,Hillingdon,4.55,6.65,2.1,religion_Sikh,London


In [135]:
df[ ( ( df['topic'].map(lambda x: x == 'religion_Sikh') ) & ( df['parent'] == 'South East' ) ) ].sort_values(['change'])

,lad,2001,2011,change,topic,parent
E06000046,Isle of Wight,0.02,0.03,0.01,religion_Sikh,South East
E06000043,Brighton and Hove,0.1,0.13,0.03,religion_Sikh,South East
E06000037,West Berkshire,0.1,0.15,0.05,religion_Sikh,South East
E06000044,Portsmouth,0.17,0.23,0.06,religion_Sikh,South East
E06000038,Reading,0.55,0.61,0.06,religion_Sikh,South East
E06000042,Milton Keynes,0.38,0.55,0.17,religion_Sikh,South East
E06000045,Southampton,1.29,1.47,0.18,religion_Sikh,South East
E06000036,Bracknell Forest,0.19,0.4,0.21,religion_Sikh,South East
E06000035,Medway,1.22,1.46,0.24,religion_Sikh,South East
E06000041,Wokingham,0.97,1.32,0.35,religion_Sikh,South East


In [136]:
# Create triple data for areas of closest proximity
geogTriples = []
for lad in lads:
    list1 = lad['bounds'][0]+lad['bounds'][1]
    for lad2 in lads:
        list2 = lad2['bounds'][0]+lad2['bounds'][1]
        listDif = [abs(list1[i]-list2[i]) for i in [0,1,2,3]]
        listDif.sort()
        if (sum(listDif[:3]) < 5) & (lad!=lad2):
            geogTriples.append([lad['name'], lad2['name'], ("near", round(sum(listDif[:3]), 2))])

In [137]:
subjectList = ["NoWelsh", "Male31-48", "Male16-30", "Female31-48", "Female16-30", "Widowed", "65andOver", "Other", "noCare", "MultiOther", "fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "female", "male", "inactive", "LoneNKids", "MultiStudents", '1to19hoursWeek', "Religionnotstated"]
topicList = ["age10yr", "travel"]

# Find each LAD local and national ranks and put in prioirity order
for thisLad in lads:

    if (areaClassDF[areaClassDF['Code']==thisLad['code']]['Subgroup code'].shape[0]>0):
        thisLad['classification'] = areaClassLU[areaClassDF[areaClassDF['Code']==thisLad['code']]['Subgroup code'].iloc[0]]
        thisLad['Subgroup code'] = areaClassDF[areaClassDF['Code']==thisLad['code']]['Subgroup code'].iloc[0]
    else:
        thisLad['classification'] = "unk"
    thisLad['gss'] = {}
    thisLad['gss']['short'] = gssLookupShort[thisLad['code'][:3]]
    thisLad['gss']['long'] = gssLookup[thisLad['code'][:3]]

    if thisLad['parents'][0]['type'] == "cty":
        thisLad['parents'] = thisLad['parents'][1:]

    # Empty array will be populted with ranks for each variable
    ranks = []

    # Filter areas with same parent
    sister_lads = []
    try:
        par_name = next(i for i in thisLad['parents'] if ([j for j in i.values()][2] == 'rgn'))['name']
    except:
        par_name = next(i for i in thisLad['parents'] if ([j for j in i.values()][2] == 'ctry'))['name']
        
    for thatLad in lads:
        try:
            if  par_name == next(i for i in thatLad['parents'] if ([j for j in i.values()][2] == 'rgn'))['name']:
                sister_lads.append(thatLad)
        except:
            if par_name=='Wales':
                sister_lads.append(thatLad)


    # CREATE THE RANKINGS
    # Loop through the various data variables
    for a in thisLad['data']:
        if a not in topicList:
            if a in ['population', 'density', 'agemed']:
                b = 'value'
            else:
                b = 'perc'
            # Create nested object with localised rank
            thisLad['data'][a][b+"_rank_local"] = {}    
            for c in ['2001', '2011', 'change']:
                thisLad['data'][a][b+"_rank_local"][c] = {}
                for d in thisLad['data'][a][b][c]:
                    if d not in subjectList:
                        vari = thisLad['data'][a][b][c][d]

                        # Create sorted list of values from sister areas
                        group_values = []
                        for lad in sister_lads:
                            group_values.append(lad['data'][a][b][c][d])
                            group_values = [x if (type(x) == float) | (type(x) == int) else np.nan for x in group_values]
                        group_values.sort(reverse=True)

                        # Find index of value of area of interest
                        varRank = group_values.index(vari) + 1

                        # Convert bottom half rankings into negative values
                        if varRank>len(group_values)/2:
                            varRank = varRank-len(group_values)-1

                        # Find the variable's rank nationally
                        if thisLad['parents'][0]['name'] == "Wales":
                            natRank = [i for i in df[(df['parent']=="Wales")&(df['topic']==a+"_"+d)].sort_values(by=[c], ascending=False)['lad']].index(thisLad['name'])+1
                            if natRank > noOfLads["Wales"]/2:
                                natRank = natRank-noOfLads["Wales"]-1
                        else:
                            # Index + 1 gives toprank = 1
                            natRank = [i for i in df[(df['parent']!="Wales")&(df['topic']==a+"_"+d)].sort_values(by=[c], ascending=False)['lad']].index(thisLad['name'])+1
                            # If the rank is in the bottom half, ie, greater value than the middle rank
                            if natRank > noOfLads["England"]/2:
                                natRank = (natRank-noOfLads["England"])-1

                        thisLad['data'][a][b+"_rank_local"][c][d] = varRank
                        thisLad['data'][a][b+"_rank"][c][d] = natRank

                        # Append ranking data to original array
                        ranks.append({
                            'label': a+'_'+b+'_'+c+'_'+d, 
                            'locRank': varRank, 
                            'natRank': natRank, 
                            'value': vari
                        })

    # Sort in rank order
    ranks = sorted(ranks, key=lambda x: (abs(x['locRank']), -abs(x['value'])))
    thisLad["Priorities"] = ranks

In [138]:
# Find each LAD find statistically similar areas
for thisLad in lads:

    nearThis = [i for i in geogTriples if i[0]==thisLad['name']]
    simiThis = list(areaClassDF[areaClassDF['Subgroup code']==thisLad['Subgroup code']].Name)
    nearandsimi = [i for i in nearThis if i[1] in simiThis]
    nearandsimi = sorted(nearandsimi, reverse=False, key=lambda x: x[2])

    try:
        thisLad['similar'] = [
            {'code': simiDF[simiDF['Code']==thisLad['code']]['Code.1'].iloc[0], 'name': simiDF[simiDF['Code']==thisLad['code']]['Name.1'].iloc[0]},
            {'code': simiDF[simiDF['Code']==thisLad['code']]['Code.2'].iloc[0], 'name': simiDF[simiDF['Code']==thisLad['code']]['Name.2'].iloc[0]},
            {'code': simiDF[simiDF['Code']==thisLad['code']]['Code.3'].iloc[0], 'name': simiDF[simiDF['Code']==thisLad['code']]['Name.3'].iloc[0]}
        ]
    except:
        thisLad['similar'] = [
            {'code': this(nearandsimi[0][1])['code'], 'name': nearandsimi[0][1]},
            {'code': this(nearandsimi[1][1])['code'], 'name': nearandsimi[1][1]}
        ]

In [139]:
# Increase a positive or negative number by 1 (-1 => -2)
def incmag(x):
    return int((abs(x)+1) * (x/abs(x)))

In [140]:
lads[9]['data']['health']['perc_rank_local']

{'2001': {'bad': -16, 'good': 12},
 '2011': {'bad': -14, 'good': 12},
 'change': {'bad': 17, 'good': -17}}

In [141]:
df = pd.DataFrame(columns=
                  ['lad', '2001',  '2011',  'change', 'natRank', 'natRank_11', 'localRank', 'localRank_11', 'topic', 'parent'])
for j in range(len(topics)):
    df1 = pd.DataFrame(index=ladCodes, columns=
                      ['lad', '2001',  '2011',  'change', 'natRank', 'natRank_11', 'localRank', 'localRank_11', 'topic', 'parent'])

    for i in range(len(lads)):
        df1['lad'].iloc[i] = lads[i]['name']
        df1['2001'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2001'][topics[j][3]]
        df1['2011'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['2011'][topics[j][3]]    
        df1['change'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]]['change'][topics[j][3]]
        df1['natRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['change'][topics[j][3]]
        df1['natRank_11'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank"]['2011'][topics[j][3]]
        df1['localRank'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['change'][topics[j][3]]    
        df1['localRank_11'].iloc[i] = lads[i]['data'][topics[j][1]][topics[j][2]+"_rank_local"]['2011'][topics[j][3]]    
        df1['topic'].iloc[i] = topics[j][1]+"_"+topics[j][3]
        try:
            df1['parent'].iloc[i] = next(k for k in lads[i]['parents'] if ([j for j in k.values()][2] == 'rgn'))['name']
        except:
            df1['parent'].iloc[i] = next(k for k in lads[i]['parents'] if ([j for j in k.values()][2] == 'ctry'))['name']

    df = pd.concat([df,df1])

for topic in topics:
    for lad in lads:
        
        # Find the areas that this area has overtaken
        v2001 = lad['data'][topic[1]][topic[2]]['2001'][topic[3]]
        v2011 = lad['data'][topic[1]][topic[2]]['2011'][topic[3]]
        if thisLad['parents'][0]['name'] == "Wales":
            dfT = df[(df['parent']=="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]
        else:
            dfT = df[(df['parent']!="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)]

        obje = []
        for i in range(dfT.shape[0]):
            obje.append(dfT.iloc[i]['lad'])
        
        if 'overtake' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank"]['overtake'][topic[3]] = obje

        # Find areas within the region that this area has overtaken
        reg = lad['parents'][0]['name']
        dfTr = df[(df['topic']==topic[1]+"_"+topic[3])&(df['2001']>v2001)&(df['2011']<v2011)&(df['parent']==reg)]
        objeR = []
        for i in range(dfTr.shape[0]):
            objeR.append(dfTr.iloc[i]['lad'])
        if 'overtake' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['overtake'] = {}
        lad['data'][topic[1]][topic[2]+"_rank_local"]['overtake'][topic[3]] = objeR
        
        dfReg = df[(df['topic']==topic[1]+"_"+topic[3])&(df['parent']==reg)]
        rank2001 = sorted(dfReg['2001'], reverse=True).index(v2001)+1
        if rank2001 > 168:
            rank2001 = rank2001-336-1
        if '2001' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['2001'] = {}
        lad['data'][topic[1]][topic[2]+'_rank_local']['2001'][topic[3]] = rank2001

        # Find the area immediatly above or below
        above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-2
        below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]
        if below<0:
            above = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]-1
            below = lad['data'][topic[1]][topic[2]+'_rank_local']['2011'][topic[3]]+1
            
        name_above = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[above]['lad']
        name_below = df[(df['parent']==reg)&(df['topic']==topic[1]+"_"+topic[3])].sort_values('2011', ascending=False).iloc[below]['lad']

        area_above = {'name': name_above,
                         'value': df[(df['lad']==name_above)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}
        area_below = {'name': name_below,
                          'value': df[(df['lad']==name_below)&(df['topic']==topic[1]+"_"+topic[3])]['2011'].iloc[0]}


        if 'above_below' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'] = {}

        if topic[3] not in lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]] = {}
        
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["above"] = area_above
        lad['data'][topic[1]][topic[2]+'_rank_local']['above_below'][topic[3]]["below"] = area_below
            
        # Add top and bottom three biggest movers for each subject to every area
        if thisLad['parents'][0]['name'] == "Wales":
            df_topic = df[(df['parent']=="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]
        else:
            df_topic = df[(df['parent']!="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank'])<4)]

        # This creates a an object of top and bottom areas based on size of change
        ob = {}
        for index, row in df_topic.iterrows():
            ob[int(row['natRank'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom'][topic[3]]=ob


        # Add top and bottom three biggest scorers for 2011 for each subject to every area
        if thisLad['parents'][0]['name'] == "Wales":
            df_topic = df[(df['parent']=="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank_11'])<4)]
        else:
            df_topic = df[(df['parent']!="Wales")&(df['topic']==topic[1]+"_"+topic[3])&(abs(df['natRank_11'])<4)]

        # This creates a an object of top and bottom areas based on size of 2011 value
        ob11 = {}
        for index, row in df_topic.iterrows():
            ob11[int(row['natRank_11'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        if 'top_bottom_11' not in lad['data'][topic[1]][topic[2]+'_rank'].keys():
            lad['data'][topic[1]][topic[2]+'_rank']['top_bottom_11'] = {}
        lad['data'][topic[1]][topic[2]+'_rank']['top_bottom_11'][topic[3]]=ob11

        # Add top and bottom three biggest movers for each subject to every area (regional)
        df_topicReg = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['localRank'])<4)&(df['parent']==reg)]

        ob = {}
        for index, row in df_topicReg.iterrows():
            if int(row['localRank']) in ob.keys():
                if int(row['localRank'])+1 in ob.keys():
                    ob[int(row['localRank'])+2] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
                else:
                    ob[int(row['localRank'])+1] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
            else:
                ob[int(row['localRank'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        if 'top_bottom' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['top_bottom'] = {}
        lad['data'][topic[1]][topic[2]+'_rank_local']['top_bottom'][topic[3]]=ob



        # Add top and bottom three biggest scorers for 2011 for each subject to every area (regional)
        df_topicReg11 = df[(df['topic']==topic[1]+"_"+topic[3])&(abs(df['localRank_11'])<4)&(df['parent']==reg)]

        # This creates a an object of top and bottom areas based on size of 2011 value
        ob11 = {}
        for index, row in df_topicReg11.iterrows():
            if int(row['localRank_11']) in ob11.keys():
                if int(row['localRank_11'])+1 in ob11.keys():
                    ob11[int(row['localRank_11'])+2] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
                else:
                    ob11[int(row['localRank_11'])+1] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
            else:
                ob11[int(row['localRank_11'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        if 'top_bottom_11' not in lad['data'][topic[1]][topic[2]+'_rank_local'].keys():
            lad['data'][topic[1]][topic[2]+'_rank_local']['top_bottom_11'] = {}
        lad['data'][topic[1]][topic[2]+'_rank_local']['top_bottom_11'][topic[3]]=ob11


In [142]:
df_topicReg = df[(df['topic']=='household_OnePerson')&(abs(df['localRank'])<4)&(df['parent']=="South East")]
ob = {}
for index, row in df_topicReg.iterrows():
    if int(row['localRank']) in ob.keys():
        if int(row['localRank'])+1 in ob.keys():
            ob[int(row['localRank'])+2] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
        else:
            ob[int(row['localRank'])+1] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}
    else:
        ob[int(row['localRank'])] = {'name': row['lad'], 2001: row['2001'], 2011: row['2011'], 'change': row['change']}

ob

{1: {'name': 'Folkestone and Hythe', 2001: 30.7, 2011: 33.33, 'change': 2.63},
 3: {'name': 'Havant', 2001: 26.56, 2011: 28.6, 'change': 2.04},
 -2: {'name': 'Guildford', 2001: 29.07, 2011: 26.72, 'change': -2.35},
 -1: {'name': 'Brighton and Hove', 2001: 39.35, 2011: 36.44, 'change': -2.91},
 2: {'name': 'Vale of White Horse', 2001: 24.18, 2011: 26.44, 'change': 2.26}}

In [143]:
df

,lad,2001,2011,change,natRank,natRank_11,localRank,localRank_11,topic,parent
W06000015,Cardiff,305353,346090,13.34,1,1,1,1,population_all,Wales
E08000027,Dudley,305155,312925,2.55,-41,29,-5,3,population_all,West Midlands
E07000241,Welwyn Hatfield,97553,110535,13.31,38,-105,8,-18,population_all,East
E07000112,Folkestone and Hythe,96238,107969,12.19,48,-99,7,-21,population_all,South East
E09000020,Kensington and Chelsea,158919,158649,-0.17,-12,116,-1,-2,population_all,London
...,...,...,...,...,...,...,...,...,...,...
E07000108,Dover,8.05,9.82,1.77,54,-144,13,-18,economic_self-employed,South East
E09000007,Camden,11.5,13.41,1.91,38,39,-10,6,economic_self-employed,London
E07000131,Harborough,11.67,12.89,1.22,-151,52,9,2,economic_self-employed,East Midlands
E08000022,North Tyneside,5.23,6.52,1.29,144,-25,6,4,economic_self-employed,North East


In [144]:
df[(df['topic']=='household_OnePerson')&(abs(df['localRank'])<5)&(df['parent']=="South East")]

,lad,2001,2011,change,natRank,natRank_11,localRank,localRank_11,topic,parent
E07000112,Folkestone and Hythe,30.7,33.33,2.63,20,43,1,8,household_OnePerson,South East
E07000090,Havant,26.56,28.6,2.04,44,-134,3,25,household_OnePerson,South East
E07000217,Woking,28.29,26.6,-1.69,-36,-52,-4,-20,household_OnePerson,South East
E07000211,Reigate and Banstead,29.05,27.36,-1.69,-37,-75,-4,-26,household_OnePerson,South East
E07000088,Gosport,27.95,29.82,1.87,55,128,4,19,household_OnePerson,South East
E07000209,Guildford,29.07,26.72,-2.35,-31,-56,-2,-23,household_OnePerson,South East
E06000043,Brighton and Hove,39.35,36.44,-2.91,-25,12,-1,2,household_OnePerson,South East
E07000180,Vale of White Horse,24.18,26.44,2.26,33,-50,2,-19,household_OnePerson,South East


In [145]:
df_topicReg

,lad,2001,2011,change,natRank,natRank_11,localRank,localRank_11,topic,parent
E07000112,Folkestone and Hythe,30.7,33.33,2.63,20,43,1,8,household_OnePerson,South East
E07000090,Havant,26.56,28.6,2.04,44,-134,3,25,household_OnePerson,South East
E07000209,Guildford,29.07,26.72,-2.35,-31,-56,-2,-23,household_OnePerson,South East
E06000043,Brighton and Hove,39.35,36.44,-2.91,-25,12,-1,2,household_OnePerson,South East
E07000180,Vale of White Horse,24.18,26.44,2.26,33,-50,2,-19,household_OnePerson,South East


In [146]:
lads_copy = lads.copy()

In [147]:
def moveethrel(li):
    li_temp = [i['label'].split("_")[0] for i in li]
    if li_temp[1] in ['religion', 'ethnicity']:

        topic = next(i for i in li_temp[2:] if i not in ['religion', 'ethnicity'])
        indi = li_temp.index(topic)
        li.insert(1, li.pop(indi))

    return li

In [148]:
# stories = [
#     {'label':'1', 'type': ['a']},
#     {'label':'1', 'type': ['b']},
#     {'label':'2', 'type': ['a']},
#     {'label':'2', 'type': ['c']},
#     ]
def combinestories(stories):
    stories2 = []
    for s in stories:
        # print('loop', s['label'])
        # print([i['label'] for i in stories2])
        if s['label'] in [i['label'] for i in stories2]:
            index = next((i for i, item in enumerate(stories2) if item['label'] == s['label']), -1)
            stories2[index]['type'] = stories2[index]['type'] + s['type']
        else:
            stories2.append(s)

    return stories2

In [149]:
lads = lads_copy.copy()
# Filter out priority list by subject
for lad in lads:
    subjectList = ["Male31-48", "Male16-30", "Female31-48", "Female16-30", "Widowed", "65andOver", "Other", "noCare", "MultiOther", "fair", "rent_free", "shared_ownership", "bicycle", "taxi", "moto", "other", "female", "male", "inactive", "LoneNKids", "MultiStudents", "Religionnotstated"]
    priorities = []
    priorities2011 = []
    for rank in lad['Priorities']:
        s=rank['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        if ((s[2] == "change") & (s[3] not in subjectList) & (((s[0]=="household")&(s[3]=="Married")) is False)):
            if (s[0]=='welsh') & (lad['parents'][0]['name'] != 'Wales'):
                pass
            else:
                rank['2011'] = [i for i in lad['Priorities'] if i['label']==s[0]+'_'+s[1]+'_2011_'+s[3]][0]['value']
                rank['perccha'] = round(100*((rank['value']+0.001)/([i for i in lad['Priorities'] if i['label']==s[0]+'_'+s[1]+'_2001_'+s[3]][0]['value']+0.001)),2)
                priorities.append(rank)
        if (s[2]!="change"):
            priorities2011.append(rank)
    lad['pri'] = priorities
    lad['pri2011'] = priorities2011

sign = lambda x: math.copysign(1, x)

for thisLad in lads:

    # Find nearby area
    nearbyArea = [i for i in geogTriples if i[0]==thisLad['name']]
    nearbyArea = [i for i in nearbyArea if this(i[1])['parents'][1]['name'] == thisLad['parents'][1]['name']]

    nearbyArea = sorted(nearbyArea, key=lambda x: x[2][1])

    #Find differences in data
    if (thisLad['parents'][0]['name']=='Wales'):
        region=[i for i in countries if i['name']=="Wales"][0]
        country=[i for i in countries if i['name']=="Wales"][0]
    else:
        region=[i for i in regions if i['name']==thisLad['parents'][0]['name']][0]
        country=[i for i in countries if i['name']==thisLad['parents'][1]['name']][0]

    def reg(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]    
        try:
            return region['data'][s[0]][s[1]][s[2]][s[3]]
        except:
            if (thisLad['parents'][0]['name']=='Wales'):
                return country['data'][s[0]][s[1]][s[2]][s[3]]
            else:
                return 0

    regDiff = [(i, {'val': reg(i), 'dif': i['value']-reg(i)}) for i in thisLad['pri']]
    regDiff = sorted(regDiff, reverse=True, key=lambda x: abs(x[1]['dif']))

    def regpc(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]    
        try:
            return round(100*( (region['data'][s[0]]['perc']['change'][s[3]] + 0.001) / (region['data'][s[0]]['perc']['2001'][s[3]] + 0.001) ), 2)
        except:
            if (thisLad['parents'][0]['name']=='Wales'):
                try:
                    return round(100*( (country['data'][s[0]]['perc']['change'][s[3]] + 0.001) / (country['data'][s[0]]['perc']['2001'][s[3]] + 0.001) ), 2)
                except:
                    return 0
            else:
                return 0

    regDiffPerc = [(i, {'val': reg(i), 'dif': i['perccha']-regpc(i)}) for i in thisLad['pri']]
    regDiffPerc = sorted(regDiff, reverse=True, key=lambda x: abs(x[1]['dif']))

    # Pick out the top stories for ethnicity based on biggest percentage change 
    try:
        ethStory = next(filter(lambda x: x[0]['label'].split("_")[0] == 'ethnicity', regDiffPerc))[0]
    except:
        pass

    # Pick out the top stories for religion based on biggest percentage change 
    try:
        relStory = next(filter(lambda x: x[0]['label'].split("_")[0] == 'religion', regDiffPerc))[0]
    except:
        pass

     # remove all the other cases of ethnicity 
    thisLad['pri'] = [i for i in thisLad['pri'] if i['label'].split("_")[0]!='ethnicity']

    if ethStory['value']>0.2:
        ethStory = [{**ethStory, **{'type':['ethrel']}}]
    else:
        ethStory = []

    # remove all the other cases of religion 
    thisLad['pri'] = [i for i in thisLad['pri'] if i['label'].split("_")[0]!='religion']

    if relStory['value']>0.2:
        relStory = [{**relStory, **{'type':['ethrel']}}]
    else:
        relStory = []

    def cou(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return country['data'][s[0]][s[1]][s[2]][s[3]]
    couDiff = [(i, {'val': cou(i), 'dif': i['value']-cou(i)}) for i in thisLad['pri']]
    couDiff = sorted(couDiff, reverse=True, key=lambda x: abs(x[1]['dif']))
    
    try:
        nearbyAreaData = {}
        nearbyAreaData['name'] = nearbyArea[0][1]
        nearbyAreaData['data'] = [lad for lad in lads if lad['name']==nearbyArea[0][1]][0]['data']
    except:
        print("Nearby data not found for:: ", thisLad['name'])
        nearbyAreaData = []
        
    try:
        similarData = {}
        if (nearbyArea[0][1]!=thisLad['similar'][0]['name']):
            try:
                similarData['name'] = thisLad['similar'][0]['name']
                similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][0]['name']][0]['data']
            except:
                try:
                    similarData['name'] = thisLad['similar'][1]['name']
                    similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][1]['name']][0]['data']
                except:
                    similarData['name'] = thisLad['similar'][2]['name']
                    similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][2]['name']][0]['data']
        else:
            try:
                similarData['name'] = thisLad['similar'][1]['name']
                similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][1]['name']][0]['data']
            except:
                similarData['name'] = thisLad['similar'][2]['name']
                similarData['data'] = [lad for lad in lads if lad['name']==thisLad['similar'][2]['name']][0]['data']

    except:
        print("Similar data not found for:: ", thisLad['name'])
        similarData = []

    def simi(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return similarData['data'][s[0]][s[1]][s[2]][s[3]]
    if len(similarData)>0:
        simiDiff = [(i, {'val': simi(i), 'dif': i['value']-simi(i)}) for i in thisLad['pri']]
        simiDiff = sorted(simiDiff, reverse=True, key=lambda x: abs(x[1]['dif']))

    def near(i):
        s=i['label'].split("_")
        if len(s)>4:
            s[3] = s[3] + "_" + s[4]
        return nearbyAreaData['data'][s[0]][s[1]][s[2]][s[3]]
    if len(nearbyAreaData)>0:
        nearDiff = [(i, {'val': near(i), 'dif': i['value']-near(i)}) for i in thisLad['pri']]
        nearDiff = sorted(nearDiff, reverse=True, key=lambda x: abs(x[1]['dif']))

    nearTops = sorted([(lad['name'], lad['pri'][0]) for lad in lads if lad['name'] in [i[1] for i in nearbyArea if i[0]==thisLad['name']]], key=lambda x: abs(x[1]['locRank']))

    ageBandChange = sorted([(i, thisLad['data']['age10yr']['value']['2011'][i]-thisLad['data']['age10yr']['value']['2001'][i]) for i in thisLad['data']['age10yr']['value']['2001'].keys()], 
           reverse=True, key=lambda x: x[1])

    equalAgeChange = [lad['name'] for 
     lad in lads if (lad['parents'][0]['name'] == thisLad['parents'][0]['name']) 
     & (lad['data']['agemed']['value']['change']['all']==thisLad['data']['agemed']['value']['change']['all'])]
    len(equalAgeChange)


    # Find and refine stories
    stories = []
    stories = stories+relStory
    stories = stories+ethStory
    if (thisLad['parents'][0]['name']=='Wales'):
        stories=stories+[{**i, **{'type':['welsh']}} for i in thisLad['pri'] if i['label'] == 'welsh_perc_change_SpeaksWelsh']
    stories=stories+[{**i, **{'type':['pop']}} for i in thisLad['pri'] if i['label'] == 'population_value_change_all']
    stories=stories+[{**i, **{'type':['size']}} for i in thisLad['pri'] if ((abs(i['value'])/abs(i['2011'])>0.2)&(abs(i['value']) > 1))]
    stories=stories+[{**i, **{'type':['locRank']}} for i in thisLad['pri'] if (abs(i['locRank']) < 4)&(abs(i['value']) > 0.5)]
    stories=stories+[{**i, **{'type':['natRank']}} for i in thisLad['pri'] if (abs(i['natRank']) < 4)&(abs(i['value']) > 0.5)]
    stories=stories+[{**i[0], **{'type':['couBuck']}} for i in couDiff if ((abs(i[1]['val'])>1) & (abs(i[0]['value'])>1) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['regBuck']}} for i in regDiff if ((abs(i[1]['val'])>1) & (abs(i[0]['value'])>1) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['nearBuck']}} for i in nearDiff if ((abs(i[1]['val'])>1) & (abs(i[0]['value'])>1) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['simiBuck']}} for i in simiDiff if ((abs(i[1]['val'])>1) & (abs(i[0]['value'])>1) & (sign(i[1]['val'])!=sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['couDiff']}} for i in couDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['regDiff']}} for i in regDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['nearDiff']}} for i in nearDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i[0], **{'type':['simiDiff']}} for i in simiDiff if ((abs(i[1]['dif'])>2) & (sign(i[1]['val'])==sign(i[0]['value'])))]
    stories=stories+[{**i, **{'type':['general']}} for i in thisLad['pri']]

    # 'stories' is a list with each variable appearing duplicated for each story type its selected by

    # # 'seen' is an empty set
    # seen = set()
    # # 'uniq' is a list of unique story labels with duplicate labels being sent to the seen set
    # uniq = [item['label'] for item in stories if item['label'] in seen or seen.add(item['label'])]
    # # listofdoubles is a list story objects with unique labels
    # listofdoubles = [[i for i in stories if i['label'] == uniqI] for uniqI in uniq]
    # # 'multi' is a list which takes each of the unique label objects, 'double', and merges with a type object which refers to a list
    # multi = [{**double[0], **{'type':   [item for sublist in [i['type'] for i in double] for item in sublist]   }} for double in listofdoubles]
    # stories = [item for item in stories if item['label'] not in [i['label'] for i in multi]]
    # stories=stories+multi

    stories = combinestories(stories)

    storiesRefined =[]
    notInc = ['density', 'age10yr', 'travel']
    for i in stories:
        if i['label'].split("_")[0] not in notInc:
            storiesRefined.append(i)
            notInc.append(i['label'].split("_")[0])

    if (thisLad['code'][0]!='W'):
        storiesRefined = sorted(storiesRefined, key=lambda x: ((topicScoresOrder[bbcRegLU[thisLad['code']]][x['label'].split('_')[0]])*(1/(1+abs(x['value'])))))
    else:
        storiesRefined = sorted(storiesRefined, key=lambda x: 1/(1+abs(x['value'])))

    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: abs(x['locRank'])<4)
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: abs(x['natRank'])<4)
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: abs(x['locRank'])<3)
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: abs(x['natRank'])<3)
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['type']=='regBuck')
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['type']=='couBuck')
    # storiesRefined = storiesRefined[:6]
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='welsh_perc_change_SpeaksWelsh')
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='agemed_value_change_all')
    storiesRefined = sorted(storiesRefined, reverse=True, key=lambda x: x['label']=='population_value_change_all')

    
    # Create a varibale 'hiRank' which holds the index of a top 5 rank at the national or reginal level to be used for an overtake story
    hiRank ={}
    hiRankNat = []
    for i in storiesRefined:
        if i['label'] not in ['population_value_change_all', 'agemed_value_change_all']:
            s=i['label'].split("_")
            if len(s)>4:
                s[3] = s[3] + "_" + s[4]
            if thisLad['data'][s[0]][s[1]+"_rank"]['2011'][s[3]]!=thisLad['data'][s[0]][s[1]+"_rank"]['2001'][s[3]]:
                hiRankNat.append(abs(thisLad['data'][s[0]][s[1]+"_rank"]['2011'][s[3]]))
            else:
                hiRankNat.append(100)
        else:
            hiRankNat.append(100)

    hiRankReg = []
    for i in storiesRefined:
        if i['label'] not in ['population_value_change_all', 'agemed_value_change_all']:
            s=i['label'].split("_")
            if len(s)>4:
                s[3] = s[3] + "_" + s[4]
            if thisLad['data'][s[0]][s[1]+"_rank_local"]['2011'][s[3]]!=thisLad['data'][s[0]][s[1]+"_rank_local"]['2001'][s[3]]:
                hiRankReg.append(abs(thisLad['data'][s[0]][s[1]+"_rank_local"]['2011'][s[3]]))
            else:
                hiRankReg.append(100)
        else:
            hiRankReg.append(100)
            
    if min(hiRankNat)<5:
        hiRank['rankIn'] = hiRankNat.index(min(hiRankNat))
        hiRank['type'] = 'nat'
    elif min(hiRankReg)<5:
        hiRank['rankIn'] = hiRankReg.index(min(hiRankReg))
        hiRank['type'] = 'reg'
    elif min(hiRankNat)<10:
        hiRank['rankIn'] = hiRankNat.index(min(hiRankNat))
        hiRank['type'] = 'nat'
    else:
        hiRank['rankIn'] = "None"
        hiRank['type'] = "None"

    ageBandChange= [i for i in ageBandChange if i[0]!='all']
    ageBandPos = sorted([i for i in ageBandChange if i[1]>0], reverse=True, key=lambda x: abs(x[1]))
    ageBandNeg = sorted([i for i in ageBandChange if i[1]<0], reverse=True, key=lambda x: abs(x[1]))

    storiesRefined = moveethrel(storiesRefined)

    # Add data to object
    thisLad['hiRank']=hiRank
    thisLad['stories'] = storiesRefined
    thisLad['similar'] = similarData
    thisLad['nearbyArea'] = {}
    thisLad['nearbyArea']['triples'] = nearbyArea
    thisLad['nearbyArea']['nearTops'] = nearTops
    thisLad['nearbyArea']['nearTops'] = nearbyAreaData
    thisLad['differences'] = {}
    thisLad['differences']['country'] = couDiff
    thisLad['differences']['region'] = regDiff
    thisLad['differences']['near'] = nearDiff
    thisLad['data']['age10yr']['absChange'] = {}
    thisLad['data']['age10yr']['absChange']['pos'] = ageBandPos
    thisLad['data']['age10yr']['absChange']['neg'] = ageBandNeg
    thisLad['data']['agemed']['value_rank_local']['equalAgeChange]'] = len(equalAgeChange)

In [150]:
# similarData['name'] = this('Somerset West and Taunton')['name']
# similarData['data'] = this('Somerset West and Taunton')['data']

# this('Dorset')['similar'] = similarData


# similarData['name'] = this('Dorset')['name']
# similarData['data'] = this('Dorset')['data']

# this('Somerset West and Taunton')['similar'] = similarData


# similarData['name'] = this('West Devon')['name']
# similarData['data'] = this('West Devon')['data']

# this('Cornwall')['similar'] = similarData


# similarData['name'] = this('Buckinghamshire')['name']
# similarData['data'] = this('Buckinghamshire')['data']

# this('East Suffolk')['similar'] = similarData


# similarData['name'] = this('East Suffolk')['name']
# similarData['data'] = this('East Suffolk')['data']

# this('Buckinghamshire')['similar'] = similarData


# similarData['name'] = this('Bournemouth, Christchurch and Poole')['name']
# similarData['data'] = this('Bournemouth, Christchurch and Poole')['data']

# this('West Suffolk')['similar'] = similarData

# similarData['name'] = this('West Suffolk')['name']
# similarData['data'] = this('West Suffolk')['data']

# this('Bournemouth, Christchurch and Poole')['similar'] = similarData

# similarData['name'] = this('North Northamptonshire')['name']
# similarData['data'] = this('North Northamptonshire')['data']

# this('West Northamptonshire')['similar'] = similarData


# similarData['name'] = this('West Northamptonshire')['name']
# similarData['data'] = this('West Northamptonshire')['data']

# this('North Northamptonshire')['similar'] = similarData

#### West Suffolk, paragraph 3 example of statistically similar

In [151]:
for region in regions:
    region['data']['welsh'] = {}
    region['data']['welsh']['perc'] = {}
    region['data']['welsh']['value'] = {}
    for k in region['data']['welsh'].keys():
        region['data']['welsh'][k]['2001'] = {}
        region['data']['welsh'][k]['2011'] = {}
        region['data']['welsh'][k]['change'] = {}
        for j in region['data']['welsh'][k].keys():
            region['data']['welsh'][k][j]['SpeaksWelsh'] = 0

In [152]:
for rgn in regions:
    try:
        with open('/Users/theojolliffe/Documents/Area Reports/census-data-transformed/json/place/'+rgn['code']+'.json', 'w') as outfile:
            json.dump(rgn, outfile)
    except:
        print("Failed: ", rgn['name'])


for rgn in regions:
    try:
        with open('/Users/theojolliffe/Downloads/sk-whats-changed 2/static/data/json/place/'+rgn['code']+'.json', 'w') as outfile:
        # with open('/Users/theojolliffe/Downloads/sk-whats-changed/static/data/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(rgn, outfile)
    except:
        print("Failed: ", rgn['name'])

In [153]:
for ctry in countries:
    try:
        with open('/Users/theojolliffe/Documents/Area Reports/census-data-transformed/json/place/'+ctry['code']+'.json', 'w') as outfile:
            json.dump(ctry, outfile)
    except:
        print("Failed: ", ctry['name'])

for ctry in countries:
    try:
        with open('/Users/theojolliffe/Downloads/sk-whats-changed 2/static/data/json/place/'+ctry['code']+'.json', 'w') as outfile:
        # with open('/Users/theojolliffe/Downloads/sk-whats-changed/static/data/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(ctry, outfile)
    except:
        print("Failed: ", ctry['name'])

In [154]:
this("City of London")['gss']['short'] = 'city'
this("City of London")['gss']['long'] = 'City of London'

In [155]:
for lad in lads:
    try:
        with open('/Users/theojolliffe/Documents/Area Reports/census-data-transformed/json/place/'+lad['code']+'.json', 'w') as outfile:
        # with open('/Users/theojolliffe/Downloads/sk-whats-changed/static/data/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(lad, outfile)
    except:
        print("Failed: ", lad['name'])

for lad in lads:
    try:
        with open('/Users/theojolliffe/Downloads/sk-whats-changed 2/static/data/json/place/'+lad['code']+'.json', 'w') as outfile:
        # with open('/Users/theojolliffe/Downloads/sk-whats-changed/static/data/json/place/'+lad['code']+'.json', 'w') as outfile:
            json.dump(lad, outfile)
    except:
        print("Failed: ", lad['name'])



In [156]:
lads[1]['data']['disability']

{'perc': {'2001': {'notDisabled': 88.52,
   'little': 7.72,
   'lot': 3.7699999999999996},
  '2011': {'notDisabled': 87.86, 'little': 7.4, 'lot': 4.74},
  'change': {'notDisabled': -0.66, 'little': -0.32, 'lot': 0.97}},
 'perc_rank': {'2001': {'notDisabled': -22, 'little': 42, 'lot': 52},
  '2011': {'notDisabled': -20, 'little': 82, 'lot': 37},
  'change': {'notDisabled': -102, 'little': -127, 'lot': 32},
  'overtake': {'notDisabled': ['North Norfolk',
    'Rother',
    'East Lindsey',
    'Malvern Hills',
    'Tendring',
    'South Hams',
    'Torbay'],
   'little': ['Broxtowe', 'South Derbyshire'],
   'lot': ['Copeland',
    'Cornwall',
    'Hyndburn',
    'Bolton',
    'Nuneaton and Bedworth',
    'Bassetlaw',
    'Salford',
    'Scarborough',
    'Newcastle upon Tyne',
    'Birmingham',
    'Stockton-on-Tees',
    'West Lancashire',
    'Kingston upon Hull',
    'Wolverhampton',
    'Barking and Dagenham',
    'North Tyneside']},
  'top_bottom': {'notDisabled': {-1: {'name': 'Isles

In [157]:
!git add .
!git commit -m "Add files"
!git push -u origin main

[main 0ac73d6c] Add files
 115 files changed, 115 insertions(+), 115 deletions(-)
Enumerating objects: 232, done.
Counting objects: 100% (232/232), done.
Delta compression using up to 16 threads
Compressing objects: 100% (119/119), done.
Writing objects: 100% (119/119), 263.57 KiB | 4.18 MiB/s, done.
Total 119 (delta 117), reused 0 (delta 0)
remote: Resolving deltas: 100% (117/117), completed with 112 local objects.
To https://github.com/theojolliffe/census-data.git
   bb5f5b46..0ac73d6c  main -> main
Branch 'main' set up to track remote branch 'main' from 'origin'.


In [1]:
!pwd

/Users/theojolliffe/Documents/census-data-transformed


In [64]:
df[df['topic'].split("_")[0] == 'care']

AttributeError: 'Series' object has no attribute 'split'